<a href="https://colab.research.google.com/github/vladana-perlic/M1_Memoire_de_recherche/blob/main/Copy_of_Camembert_masquage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentencepiece
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import CamembertTokenizer
from transformers import CamembertForMaskedLM
import torch

In [ ]:
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertForMaskedLM.from_pretrained("camembert-base")
model.eval()

def fill_mask(masked_input, model, tokenizer, topk=5):
    # Adapted from https://github.com/pytorch/fairseq/blob/master/fairseq/models/roberta/hub_interface.py
    assert masked_input.count("<mask>") == 1
    input_ids = torch.tensor(tokenizer.encode(masked_input, add_special_tokens=True)).unsqueeze(0)  # Batch size 1
    logits = model(input_ids)[0]  # The last hidden-state is the first element of the output tuple
    masked_index = (input_ids.squeeze() == tokenizer.mask_token_id).nonzero().item()
    logits = logits[0, masked_index, :]
    prob = logits.softmax(dim=0)
    values, indices = prob.topk(k=topk, dim=0)
    topk_predicted_token_bpe = " ".join(
        [tokenizer.convert_ids_to_tokens(indices[i].item()) for i in range(len(indices))]
    )
    masked_token = tokenizer.mask_token
    topk_filled_outputs = []
    for index, predicted_token_bpe in enumerate(topk_predicted_token_bpe.split(" ")):
        predicted_token = predicted_token_bpe.replace("\u2581", " ")
        if " {0}".format(masked_token) in masked_input:
            topk_filled_outputs.append(
                (
                    masked_input.replace(" {0}".format(masked_token), predicted_token),
                    values[index].item(),
                    predicted_token,
                )
            )
        else:
            topk_filled_outputs.append(
                (masked_input.replace(masked_token, predicted_token), values[index].item(), predicted_token,)
            )
    return topk_filled_outputs

## Code à modifier qu'à partir d'ici

## **EXPORTER LES VALEURS DANS UN FICHIER EXCEL**

In [ ]:
import pandas as pd
from google.colab import files

# importer le fichier Excel
url = 'https://docs.google.com/spreadsheets/d/1XwPNkpb2YDUojbYYbCE_F4Un-40RyfgC/export?format=xlsx'
df = pd.read_excel(url)

cam_vocab = tokenizer.get_vocab()
# remplacer le mot par "<mask>" en fonction du statut
for i, row in df.iterrows():
    if row['classification'] in 'ajouté' and row['langue'] in 'fr':
        if row["mot ajouté --> supprimé --> modifié"] not in cam_vocab:
          continue
        masked_input = row["requête reformulée"].replace(row["mot ajouté --> supprimé --> modifié"], "<mask>")
        found = False
        topk = 100
        while not found and topk <= 30000:
            results = fill_mask(masked_input, model, tokenizer, topk=topk)
            for result in results:
                if row["mot ajouté --> supprimé --> modifié"] in result[2]:
                    found = True
                    df.loc[i, 'topk'] = topk
                    break
            topk *= 2 # augmenter topk de façon exponentielle
        if not found:
            df.loc[i, 'topk'] = "NA"
    elif row['classification'] in 'supprimé' and row['langue'] in 'fr':
        if row["mot ajouté --> supprimé --> modifié"] not in cam_vocab:
            continue
        masked_input = row["requête originale"].replace(row["mot ajouté --> supprimé --> modifié"], "<mask>")
        found = False
        topk = 100
        while not found and topk <= 30000:
            results = fill_mask(masked_input, model, tokenizer, topk=topk)
            for result in results:
                if row["mot ajouté --> supprimé --> modifié"] in result[2]:
                    found = True
                    df.loc[i, 'topk'] = topk
                    break
            topk *= 2 # augmenter topk de façon exponentielle
        if not found:
            df.loc[i, 'topk'] = "NA"

# exporter les données mises à jour dans un fichier Excel local
filename = "output.xlsx"
df.to_excel(filename, index=False, engine='openpyxl')

# télécharger le fichier Excel
files.download(filename)


KeyboardInterrupt: ignored

# **VERSION 2**

In [ ]:
import pandas as pd
from google.colab import files

# importer le fichier Excel
url = 'https://docs.google.com/spreadsheets/d/1XwPNkpb2YDUojbYYbCE_F4Un-40RyfgC/export?format=xlsx'
df = pd.read_excel(url)

cam_vocab = tokenizer.get_vocab()
# remplacer le mot par "<mask>" en fonction du statut
for i, row in df.iterrows():
    if row['mot ajouté --> supprimé --> modifié'] in cam_vocab:
        if row['classification'] == 'ajouté':
            masked_input = row['requête reformulée'].replace(row['mot ajouté --> supprimé --> modifié'], '<mask>')
            topk = 32005
            results = fill_mask(masked_input, model, tokenizer, topk=topk)
            for index, result in enumerate(results):
                if row["mot ajouté --> supprimé --> modifié"] in result[2]:
                    df.loc[i, 'topk'] = index
                    break
                else:
                    df.loc[i, 'topk'] = "NA"
        elif row['classification'] == 'supprimé':
            masked_input = row["requête originale"].replace(row["mot ajouté --> supprimé --> modifié"], "<mask>")
            topk = 32005
            results = fill_mask(masked_input, model, tokenizer, topk=topk)
            for index, result in enumerate(results):
                if row["mot ajouté --> supprimé --> modifié"] in result[2]:
                    df.loc[i, 'topk'] = index
                    break
                else:
                    df.loc[i, 'topk'] = "NA"
        elif row['classification'] == 'modifié':
            mot2 = row["mot ajouté --> supprimé --> modifié"].split(" --> ")[1]
            if mot2 in cam_vocab:
                masked_input = row["requête originale"].replace(row["mot ajouté --> supprimé --> modifié"], "<mask>")
                topk = 32005
                results = fill_mask(masked_input, model, tokenizer, topk=topk)
                for index, result in enumerate(results):
                    if mot2 in result[2]:
                        df.loc[i, 'topk'] = index
                        break
                    else:
                        df.loc[i, 'topk'] = "NA"

# exporter les données mises à jour dans un fichier Excel local
filename = "output.xlsx"
df.to_excel(filename, index=False, engine='openpyxl')

# télécharger le fichier Excel
files.download(filename)

KeyboardInterrupt: ignored

In [ ]:
print(tokenizer.vocab_size)

version 3

In [ ]:
import pandas as pd
from google.colab import files

# Import the Excel file
url = 'https://docs.google.com/spreadsheets/d/1XwPNkpb2YDUojbYYbCE_F4Un-40RyfgC/export?format=xlsx'
df = pd.read_excel(url)

cam_vocab = tokenizer.get_vocab()
# Replace the word with "<mask>" based on the status
for i, row in df.iterrows():
    word = row['mot ajouté --> supprimé --> modifié']
    if word in cam_vocab:
        if row['classification'] == 'ajouté':
            masked_input = row['requête reformulée'].replace(word, '<mask>')
            topk = 32005
            if masked_input.count("<mask>") == 1:  # Check for one occurrence of <mask>
                results = fill_mask(masked_input, model, tokenizer, topk=topk)
                for index, result in enumerate(results):
                    if word in result[2]:
                        df.loc[i, 'topk'] = index
                        break
                else:
                    df.loc[i, 'topk'] = "NA"
        elif row['classification'] == 'supprimé':
            masked_input = row["requête originale"].replace(word, "<mask>")
            topk = 32005
            if masked_input.count("<mask>") == 1:  # Check for one occurrence of <mask>
                results = fill_mask(masked_input, model, tokenizer, topk=topk)
                for index, result in enumerate(results):
                    if word in result[2]:
                        df.loc[i, 'topk'] = index
                        break
                else:
                    df.loc[i, 'topk'] = "NA"
        elif row['classification'] == 'modifié':
            word2 = word.split(" --> ")[1]
            if word2 in cam_vocab:
                masked_input = row["requête originale"].replace(word, "<mask>")
                topk = 32005
                if masked_input.count("<mask>") == 1:  # Check for one occurrence of <mask>
                    results = fill_mask(masked_input, model, tokenizer, topk=topk)
                    for index, result in enumerate(results):
                        if word2 in result[2]:
                            df.loc[i, 'topk'] = index
                            break
                    else:
                        df.loc[i, 'topk'] = "NA"
    else:
        df.loc[i, 'topk'] = "NA"

# Export the updated data to a local Excel file
filename = "output.xlsx"
df.to_excel(filename, index=False, engine='openpyxl')

# Download the Excel file
files.download(filename)


**version 4** - jedina koja valja

In [ ]:
import pandas as pd
from google.colab import files

# Import the Excel file
url = 'https://docs.google.com/spreadsheets/d/1XwPNkpb2YDUojbYYbCE_F4Un-40RyfgC/export?format=xlsx'
df = pd.read_excel(url)

cam_vocab = tokenizer.get_vocab()
# Replace the word with "<mask>" based on the status
for i, row in df.iterrows():
    word_info = str(row['mot ajouté --> supprimé --> modifié']).split(" --> ")
    word = word_info[0]
    if word in cam_vocab:
        if row['classification'] == 'ajouté':
            masked_input = row['requête reformulée'].replace(word, '<mask>')
            topk = 32005
            if masked_input.count("<mask>") == 1:  # Check for one occurrence of <mask>
                results = fill_mask(masked_input, model, tokenizer, topk=topk)
                for index, result in enumerate(results):
                    if word in result[2]:
                        df.loc[i, 'topk'] = index
                        break
                else:
                    df.loc[i, 'topk'] = "NA"
        elif row['classification'] == 'supprimé':
            masked_input = row["requête originale"].replace(word, "<mask>")
            topk = 32005
            if masked_input.count("<mask>") == 1:  # Check for one occurrence of <mask>
                results = fill_mask(masked_input, model, tokenizer, topk=topk)
                for index, result in enumerate(results):
                    if word in result[2]:
                        df.loc[i, 'topk'] = index
                        break
                else:
                    df.loc[i, 'topk'] = "NA"
        elif row['classification'] == 'modifié':
            word2 = word_info[1] if len(word_info) > 1 else ""
            if word2 in cam_vocab:
                masked_input = row["requête reformulée"].replace(word2, "<mask>")
                topk = 32005
                if masked_input.count("<mask>") == 1:  # Check for one occurrence of <mask>
                    results = fill_mask(masked_input, model, tokenizer, topk=topk)
                    for index, result in enumerate(results):
                        if word2 in result[2]:
                            df.loc[i, 'topk'] = index
                            break
                    else:
                        df.loc[i, 'topk'] = "NA"
                else:
                    df.loc[i, 'topk'] = "NA"
            else:
                df.loc[i, 'topk'] = "NA"
    else:
        df.loc[i, 'topk'] = "NA"

# Export the updated data to a local Excel file
filename = "output.xlsx"
df.to_excel(filename, index=False, engine='openpyxl')

# Download the Excel file
files.download(filename)


**version 5 - caps insensitive**

In [ ]:
import pandas as pd
from google.colab import files

# Import the Excel file
url = 'https://docs.google.com/spreadsheets/d/1QBuj7-Ie0ogBE-9_LRcm0Jnupb6YxWKN6CASFI5spss/export?format=xlsx'
df = pd.read_excel(url)

cam_vocab_lower = {k.lower(): v for k, v in tokenizer.get_vocab().items()}  # Convert model's vocabulary to lowercase

# Replace the word with "<mask>" based on the status
for i, row in df.iterrows():
    word_info = str(row['mot ajouté --> supprimé --> modifié']).lower().split(" --> ")
    word = word_info[0]
    if word in cam_vocab_lower:
        if row['classification'].lower() == 'ajouté':
            masked_input = row['requête reformulée'].lower().replace(word.lower(), '<mask>')  # Convert masked_input to lowercase
            topk = 32005
            if masked_input.count("<mask>") == 1:  # Check for one occurrence of <mask>
                results = fill_mask(masked_input, model, tokenizer, topk=topk)
                for index, result in enumerate(results):
                    if word.lower() in result[2].lower():
                        df.loc[i, 'topk'] = index
                        break
                else:
                    df.loc[i, 'topk'] = "NA"
        elif row['classification'].lower() == 'supprimé':
            masked_input = row["requête originale"].lower().replace(word.lower(), "<mask>")  # Convert masked_input to lowercase
            topk = 32005
            if masked_input.count("<mask>") == 1:  # Check for one occurrence of <mask>
                results = fill_mask(masked_input, model, tokenizer, topk=topk)
                for index, result in enumerate(results):
                    if word.lower() in result[2].lower():
                        df.loc[i, 'topk'] = index
                        break
                else:
                    df.loc[i, 'topk'] = "NA"
        elif row['classification'].lower() == 'modifié':
            word2 = word_info[1] if len(word_info) > 1 else ""
            if word2.lower() in cam_vocab_lower:
                masked_input = row["requête reformulée"].lower().replace(word2.lower(), "<mask>")  # Convert masked_input to lowercase
                topk = 32005
                if masked_input.count("<mask>") == 1:  # Check for one occurrence of <mask>
                    results = fill_mask(masked_input, model, tokenizer, topk=topk)
                    for index, result in enumerate(results):
                        if word2.lower() in result[2].lower():
                            df.loc[i, 'topk'] = index
                            break
                    else:
                        df.loc[i, 'topk'] = "NA"
                else:
                    df.loc[i, 'topk'] = "NA"
            else:
                df.loc[i, 'topk'] = "NA"
    else:
        df.loc[i, 'topk'] = "NA"

# Export the updated data to a local Excel file
filename = "output.xlsx"
df.to_excel(filename, index=False, engine='openpyxl')

# Download the Excel file
files.download(filename)


# EXEMPLE ORIGINAL :

In [ ]:
from google.colab import files
masked_input = 'expérience psychologie <mask> sur l'

# Get the results
results = [i[2] for i in fill_mask(masked_input, model, tokenizer, topk=32005)]

# Export the results to a text file
filename = "result.txt"
with open(filename, "w") as file:
    for result in results:
        file.write(result + "\n")

# Download the text file
files.download(filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#utiliser mask pour remplacer le mot cherché (donc ajouté ou supprimé)

masked_input = "collège de <mask> addictologie"
[i[2] for i in fill_mask(masked_input, model, tokenizer, topk=32005)]
# print(fill_mask(masked_input, model, tokenizer, topk=32005))

NameError: ignored

In [ ]:
#permet de voir comment sont tokenisés les mots, si +1 token alors le mot est absent dans le voc
sentences = ["chat", "insomniaque", "paradigmatique"]
for sent in sentences:
    print('Original: ', sent)
    print('Tokenized Camembert: ', tokenizer.tokenize(sent))

In [ ]:
#teste directement pour un mot sa présence dans le voc, si mot pas présent message "false" sinon "true"
cam_vocab = tokenizer.get_vocab()
if "n-grammes" not in cam_vocab:
  print("false")
else :
  print("true")


# for v in ["insomniaque"]:
#     print(cam_vocab[v],end = " ")

false
